In [32]:
import boto3
import pandas as pd
from io import StringIO
import os, glob
from datetime import datetime, timedelta

In [34]:
from io import BytesIO

In [2]:
arg_date = '2022-01-04'

In [3]:
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

In [4]:
arg_date_dt

datetime.date(2022, 1, 3)

In [5]:
s3 = boto3.resource('s3')
bucket= s3.Bucket('demo-s3-nhhung2')

In [6]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[1], '%Y-%m-%d').date() >= arg_date_dt]

In [90]:
# for obj in bucket.objects.all():
#     if datetime.strptime(obj.key.split("/")[1], '%Y-%m-%d').date() >= arg_date_dt:
#         print(obj)

In [92]:
# for obj in bucket.objects.all():
#     if datetime.strptime(obj.key.split("/")[1], '%Y-%m-%d').date() >= arg_date_dt:
#         print(obj.key.split("/")[1])


In [7]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')

In [8]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [13]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = df_all._append(df, ignore_index=True)

C:\Users\acer\AppData\Local\Temp\ipykernel_15820\2422102807.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = df_all._append(df, ignore_index=True)


In [14]:
columns = ["ISIN", "Mnemonic", "Date", "Time", "StartPrice", "EndPrice", "MinPrice", "MaxPrice", "TradedVolume"]
df_all = df_all.loc[:, columns]

In [15]:
df_all.dropna(inplace=True)

Get opening price per ISIN and day

In [16]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [17]:
df_all

,ISIN,Mnemonic,Date,Time,StartPrice,EndPrice,MinPrice,MaxPrice,TradedVolume,opening_price
0,AT0000A0E9W5,SANT,2022-01-03,08:00,14.760,14.750,14.750,14.760,4414,14.760
1,DE000A0DJ6J9,S92,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649,37.640
2,DE000A0D6554,NDX1,2022-01-03,08:00,13.990,13.960,13.940,14.030,23011,13.990
3,DE000A0D9PT0,MTX,2022-01-03,08:00,180.000,179.500,179.500,180.050,2308,180.000
4,DE000A0HN5C6,DWNI,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897,37.280
...,...,...,...,...,...,...,...,...,...,...
2035814,DE000A1J5RX9,O2D,2022-01-25,16:44,2.437,2.437,2.437,2.437,20000,2.579
2035815,DE0006204407,KU2,2022-01-25,16:44,72.800,72.800,72.800,72.800,159,73.400
2035816,DE0008232125,LHA,2022-01-25,16:44,6.677,6.677,6.677,6.677,440,6.610
2035817,US88160R1014,TL0,2022-01-25,16:44,809.000,809.000,809.000,809.000,8,804.000


In [19]:
df_all[df_all['ISIN']=='DE000A0D6554']

,ISIN,Mnemonic,Date,Time,StartPrice,EndPrice,MinPrice,MaxPrice,TradedVolume,opening_price
2,DE000A0D6554,NDX1,2022-01-03,08:00,13.99,13.96,13.94,14.03,23011,13.99
159,DE000A0D6554,NDX1,2022-01-03,08:01,13.96,13.99,13.95,13.99,2737,13.99
274,DE000A0D6554,NDX1,2022-01-03,08:02,13.97,13.98,13.97,13.98,634,13.99
575,DE000A0D6554,NDX1,2022-01-03,08:03,14.01,14.01,14.01,14.01,28,13.99
731,DE000A0D6554,NDX1,2022-01-03,08:04,14.02,14.06,14.02,14.06,1292,13.99
...,...,...,...,...,...,...,...,...,...,...
2032092,DE000A0D6554,NDX1,2022-01-25,16:29,12.49,12.48,12.48,12.50,3561,12.79
2032651,DE000A0D6554,NDX1,2022-01-25,16:35,12.50,12.50,12.50,12.50,228968,12.79
2035732,DE000A0D6554,NDX1,2022-01-25,16:37,12.50,12.50,12.50,12.50,5000,12.79
2035745,DE000A0D6554,NDX1,2022-01-25,16:38,12.50,12.50,12.50,12.50,132,12.79


Get closing price per ISIN and day

In [20]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [21]:
df_all[df_all['ISIN']=='DE000A0D6554']

,ISIN,Mnemonic,Date,Time,StartPrice,EndPrice,MinPrice,MaxPrice,TradedVolume,opening_price,closing_price
2,DE000A0D6554,NDX1,2022-01-03,08:00,13.99,13.96,13.94,14.03,23011,13.99,14.47
159,DE000A0D6554,NDX1,2022-01-03,08:01,13.96,13.99,13.95,13.99,2737,13.99,14.47
274,DE000A0D6554,NDX1,2022-01-03,08:02,13.97,13.98,13.97,13.98,634,13.99,14.47
575,DE000A0D6554,NDX1,2022-01-03,08:03,14.01,14.01,14.01,14.01,28,13.99,14.47
731,DE000A0D6554,NDX1,2022-01-03,08:04,14.02,14.06,14.02,14.06,1292,13.99,14.47
...,...,...,...,...,...,...,...,...,...,...,...
2032092,DE000A0D6554,NDX1,2022-01-25,16:29,12.49,12.48,12.48,12.50,3561,12.79,12.50
2032651,DE000A0D6554,NDX1,2022-01-25,16:35,12.50,12.50,12.50,12.50,228968,12.79,12.50
2035732,DE000A0D6554,NDX1,2022-01-25,16:37,12.50,12.50,12.50,12.50,5000,12.79,12.50
2035745,DE000A0D6554,NDX1,2022-01-25,16:38,12.50,12.50,12.50,12.50,132,12.79,12.50


In [ ]:
## group by index and date => create column opening_price_eur (lay gia tri min cho gia tri group by do, tuong tu doi voi cac cot khac ...)

In [22]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [23]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [24]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [25]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [26]:
df_all = df_all.round(decimals=2)

In [27]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13
...,...,...,...,...,...,...,...,...
54309,XS2376095068,2022-01-19,36.29,36.76,36.29,37.09,0,0.80
54310,XS2376095068,2022-01-20,36.89,38.13,36.89,38.13,54,3.74
54311,XS2376095068,2022-01-21,34.55,34.14,33.98,34.55,1857,-10.47
54312,XS2376095068,2022-01-24,30.94,30.26,29.39,31.07,5750,-11.35


In [28]:
df_all.reset_index(inplace=True)

In [31]:
df_all = df_all[df_all.Date >= arg_date]

In [26]:
arg_date


'2022-01-04'

In [29]:
df_all

,index,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
2,2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
3,3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
4,4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13
...,...,...,...,...,...,...,...,...,...
54309,54309,XS2376095068,2022-01-19,36.29,36.76,36.29,37.09,0,0.80
54310,54310,XS2376095068,2022-01-20,36.89,38.13,36.89,38.13,54,3.74
54311,54311,XS2376095068,2022-01-21,34.55,34.14,33.98,34.55,1857,-10.47
54312,54312,XS2376095068,2022-01-24,30.94,30.26,29.39,31.07,5750,-11.35


WRITE TO S3

In [30]:
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [35]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket('xetra-112233')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='xetra-112233', key='xetra_daily_report_20231216_105737.parquet')

Reading the upload File

In [36]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20231216_105737.parquet


In [37]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20231216_105737.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [38]:
df_report

,index,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
2,2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
3,3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
4,4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13
...,...,...,...,...,...,...,...,...,...
54309,54309,XS2376095068,2022-01-19,36.29,36.76,36.29,37.09,0,0.80
54310,54310,XS2376095068,2022-01-20,36.89,38.13,36.89,38.13,54,3.74
54311,54311,XS2376095068,2022-01-21,34.55,34.14,33.98,34.55,1857,-10.47
54312,54312,XS2376095068,2022-01-24,30.94,30.26,29.39,31.07,5750,-11.35
